# Predict household income from satellite imagery data

First pass.

General ML pipeline steps:
1. Import data
2. Split data into test/train sets
3. Preprocess test/train sets separately
4. Generate features from data
5. For each regressor-hyperparameter combination:
    - Train regressor with given hyperparameters and training data and labels
    - Generate predicted labels for test data with trained regressor
    - Evaluate regressor-hyperparameter performance against actual test labels and get $R^2$
6. Explore best-performing models

In [1]:
import os
import math
import pickle
import numpy as np
import pandas as pd 

from sklearn import preprocessing

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import LinearSVR
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier

from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score

import matplotlib.pyplot as plt

# Import configuration file
import config as cf

# Display options 
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = -1

# Turn off big pink warnings
import warnings
warnings.filterwarnings('ignore')

# Data file path 
final_data_file_path = "/Users/robmarty/Dropbox/World Bank/IEs/Pakistan Poverty Estimation from Satellites/Data/FinalData"
#


In [2]:
# Test grid to make sure everything works - limited models and parameters
# 'BaggingClassifier'

GRID_TEST_CLASS = {
    'regressors': ['AdaBoostClassifier',
                   'KNeighborsClassifier', 
                   'RandomForestClassifier', 
                   'GradientBoostingClassifier', 
                   'LinearSVC',
                   'SVC', 
                   'DecisionTreeClassifier'],
    'KNeighborsClassifier': [
        {'n_neighbors': n_neighbors} \
        for n_neighbors in (2,5,10,15,) 
    ],
    'LinearSVC': [
        {'penalty': penalty, 'C': C, 'loss': loss, 'max_iter': max_iter,
        'random_state': 0} \
        for penalty in ('l2', ) \
        for C in (1e-2,1,2) \
        for loss in ('epsilon_insensitive','squared_hinge', ) \
        for max_iter in (1e1, )
    ],
    'SVC': [
        {'kernel': kernel, 'C': C, 'class_weight': class_weight,         
        'random_state': 0} \
        for C in (1e-2,1,2) \
        for class_weight in (None, 'balanced',) \
        for kernel in ('linear','poly','rbf','sigmoid', ) \
    ],
    'DecisionTreeClassifier': [
        {'criterion': criterion, 'splitter': splitter, 'max_depth': max_depth,
        'max_features': max_features, 'random_state': 0} \
        for criterion in ('gini', ) \
        for splitter in ('best', ) \
        for max_depth in (1,2,3,4, 5, 10, 20, 30, 50, 70, 100, ) \
        for max_features in ('sqrt', ) \
    ],
    'BaggingClassifier': [
        {'n_estimators': n_estimators, 'max_features': max_features,
        'random_state': 0, 'n_jobs': -1} \
        for n_estimators in (10, 50, 100, 1000,) \
        for max_features in (0.1, 0.2, 0.3,0.4, 0.5, 1.0,)
    ],
    'AdaBoostClassifier': [
        {'n_estimators': n_estimators, 
         'base_estimator': base_estimator,
        'random_state': 0} \
        for n_estimators in (5, 10, 50, 100) \
        for base_estimator in (None, 
                                DecisionTreeClassifier(max_depth=2), 
                                DecisionTreeClassifier(max_depth=5),
                              DecisionTreeClassifier(max_depth=6),
                              DecisionTreeClassifier(max_depth=10),
                              DecisionTreeClassifier(max_depth=15))
    ],
    'RandomForestClassifier': [
        {'n_estimators': n_estimators, 'criterion': criterion,
        'max_depth': max_depth, 'max_features': max_features, 'n_jobs': -1,
        'random_state': 0} \
        for n_estimators in (5, 10, 100, 1000, 5000) \
        for criterion in ('gini', ) \
        for max_depth in (1,2,3,4,5,6,7,8,9,10, ) \
        for max_features in ('sqrt','log2',None, )
    ],
    'GradientBoostingClassifier': [
        {'loss': loss, 'learning_rate': rate, 'n_estimators': n_estimators,
        'criterion': criterion, 'max_features': max_features,
        'random_state': 0} \
        for loss in ('deviance', ) \
        for rate in (1e-4, )
        for n_estimators in (100, ) \
        for criterion in ('friedman_mse', ) \
        for max_features in ('sqrt', ) \
    ]
}

## 1. Import data and drop "future" rows

In [3]:
#### Predict Changes
#DATA_PATH = os.path.join(final_data_file_path, 'BISP','Merged Datasets', 'bisp_socioeconomic_satellite_firstdiff_r13.csv')
#DATA_PATH = os.path.join('/Users/robmarty/Desktop/', 'bisp_socioeconomic_satellite_firstdiff_r13.csv')

#### Predict Levels
DATA_PATH = os.path.join(final_data_file_path, 'BISP','Merged Datasets', 'bisp_socioeconomic_satellite_panel_full_satPovNAsRemoved_1hh.csv')

df = pd.read_csv(DATA_PATH)
df.shape

(4528, 502)

In [4]:
df.head()

,uid,survey_round,province,psu,locality,period,treatment,panel,present11,present13,present16,hh_size,income_last_month_N_NAs,income_last_month,income_last_year_N_NAs,income_last_year,pscores,year,b1_buff_0.1km_mean,b2_buff_0.1km_mean,b3_buff_0.1km_mean,b4_buff_0.1km_mean,b5_buff_0.1km_mean,b6_buff_0.1km_mean,b7_buff_0.1km_mean,b12_buff_0.1km_mean,b13_buff_0.1km_mean,b14_buff_0.1km_mean,b15_buff_0.1km_mean,b16_buff_0.1km_mean,b17_buff_0.1km_mean,b23_buff_0.1km_mean,b24_buff_0.1km_mean,b25_buff_0.1km_mean,b26_buff_0.1km_mean,b27_buff_0.1km_mean,b34_buff_0.1km_mean,b35_buff_0.1km_mean,b36_buff_0.1km_mean,b37_buff_0.1km_mean,b45_buff_0.1km_mean,b46_buff_0.1km_mean,b47_buff_0.1km_mean,b56_buff_0.1km_mean,b57_buff_0.1km_mean,b67_buff_0.1km_mean,b1_buff_0.5km_mean,b2_buff_0.5km_mean,b3_buff_0.5km_mean,b4_buff_0.5km_mean,b5_buff_0.5km_mean,b6_buff_0.5km_mean,b7_buff_0.5km_mean,b12_buff_0.5km_mean,b13_buff_0.5km_mean,b14_buff_0.5km_mean,b15_buff_0.5km_mean,b16_buff_0.5km_mean,b17_buff_0.5km_mean,b23_buff_0.5km_mean,b24_buff_0.5km_mean,b25_buff_0.5km_mean,b26_buff_0.5km_mean,b27_buff_0.5km_mean,b34_buff_0.5km_mean,b35_buff_0.5km_mean,b36_buff_0.5km_mean,b37_buff_0.5km_mean,b45_buff_0.5km_mean,b46_buff_0.5km_mean,b47_buff_0.5km_mean,b56_buff_0.5km_mean,b57_buff_0.5km_mean,b67_buff_0.5km_mean,b1_buff_1km_mean,b2_buff_1km_mean,b3_buff_1km_mean,b4_buff_1km_mean,b5_buff_1km_mean,b6_buff_1km_mean,b7_buff_1km_mean,b12_buff_1km_mean,b13_buff_1km_mean,b14_buff_1km_mean,b15_buff_1km_mean,b16_buff_1km_mean,b17_buff_1km_mean,b23_buff_1km_mean,b24_buff_1km_mean,b25_buff_1km_mean,b26_buff_1km_mean,b27_buff_1km_mean,b34_buff_1km_mean,b35_buff_1km_mean,b36_buff_1km_mean,b37_buff_1km_mean,b45_buff_1km_mean,b46_buff_1km_mean,b47_buff_1km_mean,b56_buff_1km_mean,b57_buff_1km_mean,b67_buff_1km_mean,b1_buff_1.5km_mean,b2_buff_1.5km_mean,b3_buff_1.5km_mean,b4_buff_1.5km_mean,b5_buff_1.5km_mean,b6_buff_1.5km_mean,b7_buff_1.5km_mean,b12_buff_1.5km_mean,b13_buff_1.5km_mean,b14_buff_1.5km_mean,b15_buff_1.5km_mean,b16_buff_1.5km_mean,b17_buff_1.5km_mean,b23_buff_1.5km_mean,b24_buff_1.5km_mean,b25_buff_1.5km_mean,b26_buff_1.5km_mean,b27_buff_1.5km_mean,b34_buff_1.5km_mean,b35_buff_1.5km_mean,b36_buff_1.5km_mean,b37_buff_1.5km_mean,b45_buff_1.5km_mean,b46_buff_1.5km_mean,b47_buff_1.5km_mean,b56_buff_1.5km_mean,b57_buff_1.5km_mean,b67_buff_1.5km_mean,b1_buff_2km_mean,b2_buff_2km_mean,b3_buff_2km_mean,b4_buff_2km_mean,b5_buff_2km_mean,b6_buff_2km_mean,b7_buff_2km_mean,b12_buff_2km_mean,b13_buff_2km_mean,b14_buff_2km_mean,b15_buff_2km_mean,b16_buff_2km_mean,b17_buff_2km_mean,b23_buff_2km_mean,b24_buff_2km_mean,b25_buff_2km_mean,b26_buff_2km_mean,b27_buff_2km_mean,b34_buff_2km_mean,b35_buff_2km_mean,b36_buff_2km_mean,b37_buff_2km_mean,b45_buff_2km_mean,b46_buff_2km_mean,b47_buff_2km_mean,b56_buff_2km_mean,b57_buff_2km_mean,b67_buff_2km_mean,b1_buff_0.1km_min,b2_buff_0.1km_min,b3_buff_0.1km_min,b4_buff_0.1km_min,b5_buff_0.1km_min,b6_buff_0.1km_min,b7_buff_0.1km_min,b12_buff_0.1km_min,b13_buff_0.1km_min,b14_buff_0.1km_min,b15_buff_0.1km_min,b16_buff_0.1km_min,b17_buff_0.1km_min,b23_buff_0.1km_min,b24_buff_0.1km_min,b25_buff_0.1km_min,b26_buff_0.1km_min,b27_buff_0.1km_min,b34_buff_0.1km_min,b35_buff_0.1km_min,b36_buff_0.1km_min,b37_buff_0.1km_min,b45_buff_0.1km_min,b46_buff_0.1km_min,b47_buff_0.1km_min,b56_buff_0.1km_min,b57_buff_0.1km_min,b67_buff_0.1km_min,b1_buff_0.5km_min,b2_buff_0.5km_min,b3_buff_0.5km_min,b4_buff_0.5km_min,b5_buff_0.5km_min,b6_buff_0.5km_min,b7_buff_0.5km_min,b12_buff_0.5km_min,b13_buff_0.5km_min,b14_buff_0.5km_min,b15_buff_0.5km_min,b16_buff_0.5km_min,b17_buff_0.5km_min,b23_buff_0.5km_min,b24_buff_0.5km_min,b25_buff_0.5km_min,b26_buff_0.5km_min,b27_buff_0.5km_min,b34_buff_0.5km_min,b35_buff_0.5km_min,b36_buff_0.5km_min,b37_buff_0.5km_min,b45_buff_0.5km_min,b46_buff_0.5km_min,b47_buff_0.5km_min,b56_buff_0.5km_min,b57_buff_0.5km_min,b67_buff_0.5km_min,b1_buff_1km_min,b2_buff_1km_min,b3_buff_1km_min,b4_buff_1km_min,b5_buff_1km_min,b6_buff_1km_min,b7_bu

In [5]:
#### Restrict to Year
#df = df[df['year'] == 2013]

In [6]:
#### Changes
#df['pscores_bin'] = df['pscores'] < 0

#### Levels
#df = df.loc[df['survey_round'] != 1]
#df['pscores_bin'] = df['pscores'] <= (df['pscores'].median())
#df['pscores_bin'] = df['pscores_poor']

# DV as Quantiles
df['pscores_bin'] = pd.qcut(df['pscores'], 4, labels=False)
#df['pscores_2011'].value_counts()
#df['pscores_bin'] = df['pscores'] < 0

df.pscores_bin.value_counts()

1    1134
0    1133
2    1132
3    1129
Name: pscores_bin, dtype: int64

In [7]:
df = df[df['pscores_bin'] != 1]
df = df[df['pscores_bin'] != 2]
df['pscores_bin'] = (df['pscores_bin'] == 0)

In [8]:
# Keep Select Columns
df_viirs = df.filter(regex='viirs').filter(regex='_2km')
df_landsat = df.filter(regex='^b').filter(regex='_1km')
df_osm = df.filter(regex='fclass').filter(regex='meters')
df_facebook = df.filter(regex='^estimate_dau')

df_y = df.filter(regex='^pscores_bin$')

df_all = df_y.join(df_viirs).join(df_landsat).join(df_osm).join(df_facebook)
df_all.head()

,pscores_bin,viirs_spatialmean_monthlymean_buff_2km,viirs_spatialmean_monthlysd_buff_2km,viirs_spatialmax_monthlymean_buff_2km,viirs_spatialmax_monthlysd_buff_2km,viirs_spatialmin_monthlymean_buff_2km,viirs_spatialmin_monthlysd_buff_2km,viirs_spatialsd_monthlymean_buff_2km,viirs_spatialsd_monthlysd_buff_2km,b1_buff_1km_mean,b2_buff_1km_mean,b3_buff_1km_mean,b4_buff_1km_mean,b5_buff_1km_mean,b6_buff_1km_mean,b7_buff_1km_mean,b12_buff_1km_mean,b13_buff_1km_mean,b14_buff_1km_mean,b15_buff_1km_mean,b16_buff_1km_mean,b17_buff_1km_mean,b23_buff_1km_mean,b24_buff_1km_mean,b25_buff_1km_mean,b26_buff_1km_mean,b27_buff_1km_mean,b34_buff_1km_mean,b35_buff_1km_mean,b36_buff_1km_mean,b37_buff_1km_mean,b45_buff_1km_mean,b46_buff_1km_mean,b47_buff_1km_mean,b56_buff_1km_mean,b57_buff_1km_mean,b67_buff_1km_mean,b1_buff_1km_min,b2_buff_1km_min,b3_buff_1km_min,b4_buff_1km_min,b5_buff_1km_min,b6_buff_1km_min,b7_buff_1km_min,b12_buff_1km_min,b13_buff_1km_min,b14_buff_1km_min,b15_buff_1km_min,b16_buff_1km_min,b17_buff_1km_min,b23_buff_1km_min,b24_buff_1km_min,b25_buff_1km_min,b26_buff_1km_min,b27_buff_1km_min,b34_buff_1km_min,b35_buff_1km_min,b36_buff_1km_min,b37_buff_1km_min,b45_buff_1km_min,b46_buff_1km_min,b47_buff_1km_min,b56_buff_1km_min,b57_buff_1km_min,b67_buff_1km_min,b1_buff_1km_max,b2_buff_1km_max,b3_buff_1km_max,b4_buff_1km_max,b5_buff_1km_max,b6_buff_1km_max,b7_buff_1km_max,b12_buff_1km_max,b13_buff_1km_max,b14_buff_1km_max,b15_buff_1km_max,b16_buff_1km_max,b17_buff_1km_max,b23_buff_1km_max,b24_buff_1km_max,b25_buff_1km_max,b26_buff_1km_max,b27_buff_1km_max,b34_buff_1km_max,b35_buff_1km_max,b36_buff_1km_max,b37_buff_1km_max,b45_buff_1km_max,b46_buff_1km_max,b47_buff_1km_max,b56_buff_1km_max,b57_buff_1km_max,b67_buff_1km_max,dist_osm_fclass_tertiary_meters,dist_osm_fclass_secondary_meters,dist_osm_fclass_residential_meters,dist_osm_fclass_trunk_meters,dist_osm_fclass_primary_meters,dist_osm_fclass_unclassified_meters,dist_osm_fclass_service_meters,dist_osm_fclass_motorway_meters,dist_osm_fclass_living_street_meters,estimate_dau_all,estimate_dau_male,estimate_dau_female
1,True,2.842941,0.437466,8.700871,1.975251,0.676076,0.176400,2.305804,0.647746,852.733208,1156.635061,1255.240012,2644.950926,2367.389548,2973.689346,1724.653445,0.151242,0.190945,0.512401,0.470372,0.554292,0.338296,0.040883,0.391499,0.343572,0.439930,0.197140,0.356319,0.307001,0.406356,0.157527,-0.055376,0.058509,-0.210613,0.113516,-0.157070,-0.265846,548.0,745.0,682.0,1401.0,1056.5,2921.0,638.0,0.152359,0.108943,0.437660,0.316921,0.684059,0.075885,-0.044149,0.305685,0.172911,0.593562,-0.077368,0.345175,0.215416,0.621427,-0.033333,-0.140183,0.351689,-0.374203,0.468762,-0.246976,-0.641472,1566.5,2042.5,2759.5,3538.0,4762.0,3049.0,4023.0,0.131892,0.275774,0.386228,0.504938,0.321200,0.439485,0.149313,0.267987,0.399662,0.197682,0.326519,0.123620,0.266237,0.049841,0.186288,0.147470,-0.074237,0.064145,-0.219306,-0.084121,0.137726,949.344371,821.615637,173.271184,135.784050,4011.442510,71.969127,2002.804245,5577.352264,254.089102,21718,16463,3696
2,True,2.725424,0.407873,8.700871,1.975251,0.599399,0.161676,2.348795,0.650181,853.438755,1166.697992,1275.906314,2675.079430,2387.659878,2975.803317,1760.362671,0.155068,0.198403,0.516262,0.473365,0.554252,0.346975,0.044710,0.392626,0.343511,0.436718,0.202819,0.354133,0.303462,0.399815,0.159556,-0.056772,0.053217,-0.206229,0.109657,-0.151228,-0.256630,548.0,755.0,682.0,1401.0,1106.0,2934.0,687.0,0.158864,0.108943,0.437660,0.337364,0.685238,0.112551,-0.050800,0.299629,0.188608,0.590675,-0.047157,0.345175,0.237136,0.622788,0.003652,-0.117671,0.353633,-0.341954,0.452475,-0.233687,-0.620547,1566.5,1884.0,2275.0,3435.0,3644.0,3049.0,4000.0,0.092016,0.184433,0.373588,0.398714,0.321200,0.437169,0.094013,0.291596,0.318379,0.236165,0.359619,0.203152,0.231289,0.145379,0.274900,0.029524,-0.059531,0.075992,-0.088899,0.046572,0.134913,835.158410,941.645909,436.039478,404.926017,4264.388544,198.401495,2137.754112,5746.224836,525.904380,21718,16463,3

In [9]:
# Drop columns where the label is missing
#df = df.loc[~pd.isnull(df['hhinc_2011'])]

#df.shape

## 2. Split data into test/train

In [10]:
LABEL = 'pscores_bin'
TEST_SIZE = 0.3

# Separate feature sets from label sets
x_df = df_all.drop(labels=[LABEL], axis=1)
y_df = df_all[LABEL]

x_df[x_df.columns] = preprocessing.scale(x_df[x_df.columns])
#x_df[x_df.columns] = preprocessing.StandardScaler().fit_transform(x_df[x_df.columns])

# Split into test and train sets for features and labels
x_train, x_test, y_train, y_test =  train_test_split(x_df, y_df, test_size=TEST_SIZE)

In [11]:
x_df

,viirs_spatialmean_monthlymean_buff_2km,viirs_spatialmean_monthlysd_buff_2km,viirs_spatialmax_monthlymean_buff_2km,viirs_spatialmax_monthlysd_buff_2km,viirs_spatialmin_monthlymean_buff_2km,viirs_spatialmin_monthlysd_buff_2km,viirs_spatialsd_monthlymean_buff_2km,viirs_spatialsd_monthlysd_buff_2km,b1_buff_1km_mean,b2_buff_1km_mean,b3_buff_1km_mean,b4_buff_1km_mean,b5_buff_1km_mean,b6_buff_1km_mean,b7_buff_1km_mean,b12_buff_1km_mean,b13_buff_1km_mean,b14_buff_1km_mean,b15_buff_1km_mean,b16_buff_1km_mean,b17_buff_1km_mean,b23_buff_1km_mean,b24_buff_1km_mean,b25_buff_1km_mean,b26_buff_1km_mean,b27_buff_1km_mean,b34_buff_1km_mean,b35_buff_1km_mean,b36_buff_1km_mean,b37_buff_1km_mean,b45_buff_1km_mean,b46_buff_1km_mean,b47_buff_1km_mean,b56_buff_1km_mean,b57_buff_1km_mean,b67_buff_1km_mean,b1_buff_1km_min,b2_buff_1km_min,b3_buff_1km_min,b4_buff_1km_min,b5_buff_1km_min,b6_buff_1km_min,b7_buff_1km_min,b12_buff_1km_min,b13_buff_1km_min,b14_buff_1km_min,b15_buff_1km_min,b16_buff_1km_min,b17_buff_1km_min,b23_buff_1km_min,b24_buff_1km_min,b25_buff_1km_min,b26_buff_1km_min,b27_buff_1km_min,b34_buff_1km_min,b35_buff_1km_min,b36_buff_1km_min,b37_buff_1km_min,b45_buff_1km_min,b46_buff_1km_min,b47_buff_1km_min,b56_buff_1km_min,b57_buff_1km_min,b67_buff_1km_min,b1_buff_1km_max,b2_buff_1km_max,b3_buff_1km_max,b4_buff_1km_max,b5_buff_1km_max,b6_buff_1km_max,b7_buff_1km_max,b12_buff_1km_max,b13_buff_1km_max,b14_buff_1km_max,b15_buff_1km_max,b16_buff_1km_max,b17_buff_1km_max,b23_buff_1km_max,b24_buff_1km_max,b25_buff_1km_max,b26_buff_1km_max,b27_buff_1km_max,b34_buff_1km_max,b35_buff_1km_max,b36_buff_1km_max,b37_buff_1km_max,b45_buff_1km_max,b46_buff_1km_max,b47_buff_1km_max,b56_buff_1km_max,b57_buff_1km_max,b67_buff_1km_max,dist_osm_fclass_tertiary_meters,dist_osm_fclass_secondary_meters,dist_osm_fclass_residential_meters,dist_osm_fclass_trunk_meters,dist_osm_fclass_primary_meters,dist_osm_fclass_unclassified_meters,dist_osm_fclass_service_meters,dist_osm_fclass_motorway_meters,dist_osm_fclass_living_street_meters,estimate_dau_all,estimate_dau_male,estimate_dau_female
1,-0.041305,-0.241939,0.121420,-0.163097,-0.128727,-0.155001,0.220774,-0.061157,-0.659503,-0.583430,-0.595656,0.069375,-0.182367,-1.183585,-0.311393,0.344930,-0.181477,0.853982,0.934028,0.620009,0.570310,-0.493531,0.809272,0.991686,0.537328,0.550331,0.787988,1.139586,0.556696,0.966918,-0.237797,-0.232381,-0.335706,0.028893,-0.418566,-0.183303,-0.637419,-0.536755,-0.537358,-0.459715,-0.271737,-1.062882,-0.308158,0.383427,-0.065948,0.239396,0.359847,0.597831,0.264766,-0.369599,0.143140,0.304473,0.477584,0.190901,0.254785,0.391134,0.515149,0.227062,0.343342,0.295232,0.134482,0.105548,-0.176402,-0.212479,0.032824,-0.098627,0.446174,0.240263,1.370115,-0.587510,0.921622,-0.452120,1.068132,0.087291,1.679988,-0.141001,1.384712,2.653964,0.232321,2.164250,-0.015147,1.925434,-0.495617,1.165755,-0.597460,0.830922,1.480688,-0.370404,1.034012,-1.459236,-0.650218,1.059514,-0.239387,-0.445802,-0.411431,-0.758547,-0.325193,-0.449951,-0.704426,-0.768989,-0.997516,-0.129044,-0.145245,-0.129374
2,-0.056334,-0.253669,0.121420,-0.163097,-0.147552,-0.167792,0.234595,-0.059431,-0.656935,-0.557884,-0.557432,0.141436,-0.153496,-1.121432,-0.263302,0.527978,-0.005603,0.898019,0.981702,0.619580,0.680421,-0.372559,0.821153,0.990655,0.508362,0.628426,0.769740,1.085242,0.509193,1.002237,-0.253451,-0.300170,-0.303260,-0.002610,-0.310753,-0.125012,-0.637419,-0.501958,-0.537358,-0.459715,-0.200032,-0.735032,-0.218772,0.539863,-0.065948,0.239396,0.432690,0.611601,0.397227,-0.483824,0.106214,0.364122,0.450375,0.303728,0.254785,0.473164,0.525389,0.328014,0.480963,0.306376,0.308633,0.032568,-0.084515,-0.104418,0.032824,-0.461370,-0.412568,-0.018621,-0.223230,-0.587510,0.889218,-1.940313,-1.112224,-0.097780,-0.388054,-0.141001,1.341171,0.166476,0.548032,0.517274,0.395505,2.620194,0.471954,0.463104,0.329438,2.744111,-0.217317,-0.102389,1.183599,0.107373,3.291589,1.030282,-0.250762,-0.436102,-0.386946,-0.746483,-0.309730,-0.4

In [12]:
x_train.head()
x_test.head()

# check that lengths match
print(len(x_train) == len(y_train))
print(len(x_test) == len(y_test))

print(y_train.value_counts())
print(y_test.value_counts())

True
True
True     805
False    778
Name: pscores_bin, dtype: int64
False    351
True     328
Name: pscores_bin, dtype: int64


### Define Training Variables

In [13]:
DAY_FEATURES = df_all.filter(regex='^b', axis=1).columns.tolist()
NIGHT_FEATURES = df_all.filter(regex='viirs', axis=1).columns.tolist()
SATELLITE_FEATURES = df_all.filter(regex='^b|viirs').columns.tolist()
NONSATELLITE_FEATURES = df_all.filter(regex='dist_osm|estimate_').columns.tolist()
ALL_FEATURES = x_df.columns.tolist()

MAIN_FEATURES = ['viirs_spatialmean_monthlymean_buff_2km',
    'viirs_spatialmean_monthlysd_buff_2km',
     'b1_buff_1km_mean',
 'b2_buff_1km_mean',
 'b3_buff_1km_mean',
 'b4_buff_1km_mean',
 'b5_buff_1km_mean',
 'b6_buff_1km_mean',
 'b7_buff_1km_mean',
 'b12_buff_1km_mean',
 'b13_buff_1km_mean',
 'b14_buff_1km_mean',
 'b15_buff_1km_mean',
 'b16_buff_1km_mean',
 'b17_buff_1km_mean',
 'b23_buff_1km_mean',
 'b24_buff_1km_mean',
 'b25_buff_1km_mean',
 'b26_buff_1km_mean',
 'b27_buff_1km_mean',
 'b34_buff_1km_mean',
 'b35_buff_1km_mean',
 'b36_buff_1km_mean',
 'b37_buff_1km_mean',
 'b45_buff_1km_mean',
 'b46_buff_1km_mean',
 'b47_buff_1km_mean',
 'b56_buff_1km_mean',
 'b57_buff_1km_mean',
 'b67_buff_1km_mean',
    'dist_osm_fclass_tertiary_meters',
 'dist_osm_fclass_secondary_meters',
 'dist_osm_fclass_residential_meters',
 'dist_osm_fclass_trunk_meters',
 'dist_osm_fclass_primary_meters',
 'dist_osm_fclass_unclassified_meters',
 'dist_osm_fclass_service_meters',
 'dist_osm_fclass_motorway_meters',
 'dist_osm_fclass_living_street_meters',
 'estimate_dau_all',
 'estimate_dau_male',
 'estimate_dau_female']


MAIN_FEATURES_LIM = ['viirs_spatialmean_monthlymean_buff_2km',
 'b12_buff_1km_mean',
 'b13_buff_1km_mean',
 'b14_buff_1km_mean',
 'b15_buff_1km_mean',
 'b16_buff_1km_mean',
 'b17_buff_1km_mean',
 'b23_buff_1km_mean',
 'b24_buff_1km_mean',
 'b25_buff_1km_mean',
 'b26_buff_1km_mean',
 'b27_buff_1km_mean',
 'b34_buff_1km_mean',
 'b35_buff_1km_mean',
 'b36_buff_1km_mean',
 'b37_buff_1km_mean',
 'b45_buff_1km_mean',
 'b46_buff_1km_mean',
 'b47_buff_1km_mean',
 'b56_buff_1km_mean',
 'b57_buff_1km_mean',
 'b67_buff_1km_mean',
    'dist_osm_fclass_tertiary_meters',
 'dist_osm_fclass_secondary_meters',
 'dist_osm_fclass_residential_meters',
 'dist_osm_fclass_trunk_meters',
 'dist_osm_fclass_primary_meters',
 'dist_osm_fclass_unclassified_meters',
 'dist_osm_fclass_service_meters',
 'dist_osm_fclass_motorway_meters',
 'dist_osm_fclass_living_street_meters',
 'estimate_dau_all']


## 5. Train and Evaluate Regressors

### 5.1 Training

In [14]:
x_all = x_test.append(x_train)
y_all = y_test.append(y_train)

In [15]:
# Define a TrainedRegressor object to hold key results information
class TrainedRegressor:
    
    def __init__(self, method, params, features, regressor):
        self.method = method
        self.params = params
        self.regressor = regressor
        self.features = features
    
    def __repr__(self):
        return f'Trained {self.method} on feature set {self.features} with params {self.params}'

In [16]:
# Use GRID_MAIN for full grid search
# parameters = cf.GRID_TEST_CLASS
parameters = GRID_TEST_CLASS

results_df = pd.DataFrame()
results_df_all = pd.DataFrame()
results_df_trainedonly_all = pd.DataFrame()

x_trainedonly_all = x_all.copy()

trained_list = []
trained_list_all = []
count = 0
# print('Training model ', end='')
for i in parameters['regressors']:
    for j in parameters[i]:
        for k in ('MAIN_FEATURES', 'MAIN_FEATURES_LIM', 'DAY_FEATURES', 'NIGHT_FEATURES', 'ALL_FEATURES', 'SATELLITE_FEATURES', 'NONSATELLITE_FEATURES'):
        
            print(f'Model {count}: Training {i} on {k} with params {str(j)}')

            # A. Train Models --------------------------
            regressor = eval(i)(**j)
            
            trained = regressor.fit(x_train[eval(k)], y_train)
            trained_list.append(TrainedRegressor(i, str(j), k, trained))
            
            # B. Results -------------------------------------
            pred_labels = trained_list[count].regressor.predict(x_test[eval(k)])

            pred_dict = {
                'regressor': trained_list[count].method,
                'features': trained_list[count].features,
                'params': trained_list[count].params,
                'accuracy_score': accuracy_score(y_true=y_test, y_pred=pred_labels),
                'average_precision_score': average_precision_score(y_test, pred_labels),
                'recall_score': recall_score(y_test, pred_labels)
            }
    
            results_df = results_df.append(pred_dict, ignore_index=True) \
                .sort_values(by='accuracy_score', ascending=False, axis=0) \
                [['regressor', 'params', 'features', 'accuracy_score','average_precision_score',
                 'recall_score']]
        
            results_df.to_csv("/Users/robmarty/Desktop/pov_results_r13.csv")
            
            x_test['y_true'] = y_test
            x_test['y_predict_' + str(count)] = pred_labels
            #x_test.to_csv(os.path.join(final_data_file_path, 'Data with Predicted Income', 'pov_opm_data_with_predictions_traineddatamodel_testdatapredict_r13.csv'))
            x_test.to_csv(os.path.join('/Users/robmarty/Desktop', 'pov_opm_data_with_predictions.csv'))

            
            
            
            
            
            
            
  
            # A. Train ------------------------------------
            # Initialize regressor, fit data, then append TrainedRegressor object to list
            # 1. Train Data
            #regressor = eval(i)(**j)
            #trained = regressor.fit(x_train[eval(k)], y_train)
            #trained_list.append(TrainedRegressor(i, str(j), k, trained))

            # 2. All Data
            #trained_all = trained
            #trained_list_all = trained_list

            
            #trained_all = regressor.fit(x_all[eval(k)], y_all)
            #trained_list_all.append(TrainedRegressor(i, str(j), k, trained_all))
            
            
            
            
            
            
            
            # B. Results -------------------------------------
            # 1. Trained Model on Test Data - - - - - - - - - -
            #pred_labels = trained_list[count].regressor.predict(x_test[eval(k)])

            #pred_dict = {
            #    'regressor': trained_list[count].method,
            #    'features': trained_list[count].features,
            #    'params': trained_list[count].params,
            #    'accuracy_score': accuracy_score(y_true=y_test, y_pred=pred_labels)        
            #}
    
            #results_df = results_df.append(pred_dict, ignore_index=True) \
            #    .sort_values(by='accuracy_score', ascending=False, axis=0) \
            #    [['regressor', 'params', 'features', 'accuracy_score']]
        
            #results_df.to_csv("/Users/robmarty/Desktop/pov_results_r13.csv")
            
            #x_test['y_true'] = y_test
            #x_test['y_predict_' + str(count)] = pred_labels
            #x_test.to_csv(os.path.join(final_data_file_path, 'Data with Predicted Income', 'pov_opm_data_with_predictions_traineddatamodel_testdatapredict_r13.csv'))
            
            
            
            
            
            
            
            # 2. Trained All Model on All Data - - - - - - - - - -
            #pred_labels_all = trained_list_all[count].regressor.predict(x_all[eval(k)])

            # Append results to dataframe and sort by R^2
            #pred_dict = {
            #    'regressor': trained_list_all[count].method,
            #    'features': trained_list_all[count].features,
            #    'params': trained_list_all[count].params,
            #    'accuracy_score': accuracy_score(y_true=y_all, y_pred=pred_labels_all)        
            #}
    
            #results_df_all = results_df_all.append(pred_dict, ignore_index=True) \
            #    .sort_values(by='accuracy_score', ascending=False, axis=0) \
            #    [['regressor', 'params', 'features', 'accuracy_score']]
        
            #results_df_all.to_csv("/Users/robmarty/Desktop/pov_results_all_r13.csv")

            # ALL
            #x_trainedonly_all['y_true'] = y_all
            #x_trainedonly_all['y_predict_' + str(count)] = trained_list_all[count].regressor.predict(x_all[eval(k)])
            #x_trainedonly_all.to_csv(os.path.join(final_data_file_path, 'Data with Predicted Income', 'pov_opm_data_with_predictions_alldatamodel_alldatapredict_r13.csv'))
            
            
            
            
            
            
            # 3. Trained Model on All Data - - - - - - - - - -
            #pred_labels_trainedonly_all = trained_list[count].regressor.predict(x_all[eval(k)])

            # Append results to dataframe and sort by R^2
            #pred_dict = {
            #    'regressor': trained_list[count].method,
            #    'features': trained_list[count].features,
            #    'params': trained_list[count].params,
            #    'accuracy_score': accuracy_score(y_true=y_all, y_pred=pred_labels_trainedonly_all)        
            #}
    
            #results_df_trainedonly_all = results_df_trainedonly_all.append(pred_dict, ignore_index=True) \
            #    .sort_values(by='accuracy_score', ascending=False, axis=0) \
            #    [['regressor', 'params', 'features', 'accuracy_score']]
        
            #results_df_trainedonly_all.to_csv("/Users/robmarty/Desktop/pov_results_trainedonly_all_r13.csv")

            # ALL
            #x_all['y_true'] = y_all
            #x_all['y_predict_' + str(count)] = trained_list[count].regressor.predict(x_all[eval(k)])
            #x_all.to_csv(os.path.join(final_data_file_path, 'Data with Predicted Income', 'pov_opm_data_with_predictions_testdatamodel_alldatapredict_r13.csv'))

            ####
            count += 1


Model 0: Training AdaBoostClassifier on MAIN_FEATURES with params {'n_estimators': 5, 'base_estimator': None, 'random_state': 0}
Model 1: Training AdaBoostClassifier on MAIN_FEATURES_LIM with params {'n_estimators': 5, 'base_estimator': None, 'random_state': 0}
Model 2: Training AdaBoostClassifier on DAY_FEATURES with params {'n_estimators': 5, 'base_estimator': None, 'random_state': 0}
Model 3: Training AdaBoostClassifier on NIGHT_FEATURES with params {'n_estimators': 5, 'base_estimator': None, 'random_state': 0}
Model 4: Training AdaBoostClassifier on ALL_FEATURES with params {'n_estimators': 5, 'base_estimator': None, 'random_state': 0}
Model 5: Training AdaBoostClassifier on SATELLITE_FEATURES with params {'n_estimators': 5, 'base_estimator': None, 'random_state': 0}
Model 6: Training AdaBoostClassifier on NONSATELLITE_FEATURES with params {'n_estimators': 5, 'base_estimator': None, 'random_state': 0}
Model 7: Training AdaBoostClassifier on MAIN_FEATURES with params {'n_estimators'

KeyboardInterrupt: 